In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cider.validation_metrics.dependencies import (
    convert_threshold_to_percentile, 
    calculate_weighted_spearmanr, 
    calculate_weighted_pearsonr,
    calculate_metrics_binary_valued_consumption,
    calculate_utility,
    calculate_rank_residuals_by_characteristic,
    calculate_demographic_parity_per_characteristic,
    calculate_independence_btwn_proxy_and_characteristic,
    calculate_precision_and_recall_independence_characteristic)
from cider.validation_metrics.schemas import ConsumptionColumn
from cider.validation_metrics.core import (
    compute_auc_roc_with_percentile_grid, 
    compute_utility_grid, 
    calculate_optimal_utility_and_cash_transfer_size_table,
    calculate_rank_residuals_table_by_characteristic,
    calculate_demographic_parity_table_per_characteristic,
    combine_tables_on_characteristic,)
import numpy as np
import pandas as pd

In [3]:
# Make synthetic data

n_rows = 1000
n_proxies = 10
synthetic_data = {}

np.random.seed(2)
synthetic_data['household_id'] = range(n_rows)
synthetic_data['groundtruth_consumption'] = np.random.rand(n_rows) * 10
for i in range(10):
    synthetic_data["proxy_consumption"] = synthetic_data['groundtruth_consumption'] + np.random.randint(size=n_rows, low=0, high=i+5)
synthetic_data['weight'] = np.random.randint(10, 100, size=n_rows)

synthetic_df = pd.DataFrame(synthetic_data)
synthetic_df

,household_id,groundtruth_consumption,proxy_consumption,weight
0,0,4.359949,13.359949,61
1,1,0.259262,8.259262,82
2,2,5.496625,13.496625,16
3,3,4.353224,6.353224,43
4,4,4.203678,17.203678,26
...,...,...,...,...
995,995,5.985047,17.985047,44
996,996,3.589201,3.589201,30
997,997,6.803915,11.803915,34
998,998,8.531998,16.531998,54


In [4]:
# Calculate Spearman's R
spearman_r = calculate_weighted_spearmanr(synthetic_df, 3)
print(f"Spearman's R: {spearman_r}")

# Calculate Pearson's R
pearson_r = calculate_weighted_pearsonr(synthetic_df, 3)
print(f"Pearson's R: {pearson_r}")

Spearman's R: 0.544
Pearson's R: 0.57


In [5]:
convert_threshold_to_percentile(2.5, synthetic_df)

np.float64(26.182947202534585)

In [6]:
calculate_metrics_binary_valued_consumption(synthetic_df, 50, 20)

,accuracy,precision,recall,true_positive_rate,false_positive_rate,auc,roc_curve,spearman_r,pearson_r
0,0.624986,0.836286,0.320644,0.320644,0.064121,0.748836,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.54,0.57


In [7]:
calculate_utility(
    synthetic_df,
    20.0, 
    ConsumptionColumn.GROUNDTRUTH,
    5000)

np.float64(-27.09044377790041)

In [8]:
compute_auc_roc_with_percentile_grid(
    synthetic_df,
    99)

,percentile,true_positive_rate,false_positive_rate,auc
0,94.0,0.954774,0.679946,0.815498
1,73.0,0.858219,0.397849,0.815464
2,72.0,0.853651,0.404403,0.810040
3,71.0,0.839095,0.411273,0.811093
4,70.0,0.827999,0.413447,0.807063
5,65.0,0.803762,0.377687,0.797996
6,64.0,0.789922,0.387551,0.787586
7,61.0,0.763131,0.382540,0.789406
8,51.0,0.666170,0.341807,0.756308
9,42.0,0.575500,0.292944,0.750353


In [9]:
cash_transfer_at_ubi_rate = 0.1 * synthetic_df['weight'].sum()
compute_utility_grid(
    synthetic_df,
    cash_transfer_amount=cash_transfer_at_ubi_rate,
    num_grid_points=9,
    constant_relative_risk_aversion=3.0)

,percentile,cash_transfer_amount_groundtruth,cash_transfer_amount_proxy,utility_groundtruth,utility_proxy
0,1.000,693423.477570,539609.542545,-510.433429,-9.130433
1,13.375,41924.741969,41415.747753,-32.568282,-3.866936
2,25.750,20777.460683,22054.339630,-16.139216,-2.802221
3,38.125,14108.444971,14482.981085,-10.582944,-2.394308
4,50.500,10640.896648,10789.052218,-8.409631,-2.287369
5,62.875,8618.959412,8572.653044,-7.723892,-2.348769
6,75.250,7222.458104,7181.041119,-7.959007,-2.479645
7,87.625,6178.906737,6190.635331,-8.760540,-2.660407
8,100.000,5447.800000,5447.800000,-9.776605,-2.870561


In [10]:
calculate_optimal_utility_and_cash_transfer_size_table(
    synthetic_df,
    cash_transfer_amount=cash_transfer_at_ubi_rate,
    num_grid_points=10,
    constant_relative_risk_aversion=3.0
)

,optimal_population_percentile,maximum_utility,optimal_transfer_size
groundtruth_consumption,67.0,-7.757662,8036.426981
proxy_consumption,56.0,-2.294791,9764.599868


In [11]:
# Fairness metrics


In [12]:
# Add characteristic for fairness analysis
allowed_gender_values = {'male', 'female', 'other'}
synthetic_df_gender = synthetic_df.copy()
synthetic_df_gender['characteristic'] = np.random.choice(list(allowed_gender_values), size=len(synthetic_df_gender))


In [13]:
results = calculate_rank_residuals_by_characteristic(
    synthetic_df_gender)

In [14]:
calculate_demographic_parity_per_characteristic(
    synthetic_df_gender,
    threshold_percentile=50)

,groundtruth_poverty_percentage,proxy_poverty_percentage,demographic_parity
characteristic,,,
female,55.899340,51.149222,-4.750118
male,46.893865,47.382285,0.488421
other,49.248237,51.835338,2.587101


In [15]:
calculate_independence_btwn_proxy_and_characteristic(
    synthetic_df_gender,
    threshold_percentile=50)

,chi2_statistic,p_value
0,84.518278,4.436997e-19


In [16]:
calculate_precision_and_recall_independence_characteristic(
    synthetic_df_gender,
    50,
    50)

,chi2_statistic,p_value
precision,7.640448,2.192289e-02
recall,127.383805,2.182520e-28


In [17]:
df, anova_f_statistic, anova_p_value = calculate_rank_residuals_table_by_characteristic(
    synthetic_df_gender)
print(f"ANOVA F-statistic: {anova_f_statistic}, p-value: {anova_p_value}")
df

ANOVA F-statistic: 1.8932391279846452, p-value: 0.15112423004132747


,mean_rank_residual,std_rank_residual
characteristic,,
female,-0.000021,0.000305
male,-0.000015,0.000296
other,0.000021,0.000308


In [18]:
calculate_demographic_parity_table_per_characteristic(
    synthetic_df_gender,
    50
)

,groundtruth_poverty_percentage,proxy_poverty_percentage,demographic_parity,population_percentage
characteristic,,,,
female,55.899340,51.149222,-4.750118,31.146518
male,46.893865,47.382285,0.488421,33.448364
other,49.248237,51.835338,2.587101,35.405118


In [19]:
combined_table, statistics = combine_tables_on_characteristic(
    synthetic_df_gender,
    50
)

In [20]:
combined_table

,mean_rank_residual,std_rank_residual,groundtruth_poverty_percentage,proxy_poverty_percentage,demographic_parity,population_percentage
characteristic,,,,,,
female,-0.000021,0.000305,55.899340,51.149222,-4.750118,31.146518
male,-0.000015,0.000296,46.893865,47.382285,0.488421,33.448364
other,0.000021,0.000308,49.248237,51.835338,2.587101,35.405118


In [21]:
statistics

,anova_f_statistic,anova_p_value,independence_chi2,independence_p_value,precision_chi2,precision_pvalue,recall_chi2,recall_pvalue
0,1.893239,0.151124,84.518278,4.436997e-19,7.640448,0.021923,127.383805,2.182520e-28
